In [1]:
import pandas as pd

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from datetime import datetime

In [2]:
#입력을 용이하게 하기 위해 원본 파일 네이밍을 임의로 변경한 파일들을 넣었습니다. 길이만 줄어들었을 뿐 원본 파일입니다.
custom = pd.read_csv("04.Custom.csv") 
master = pd.read_csv("06.Master.csv")
broduct = pd.read_csv("01.Pruduct.csv", low_memory=False)
search1 = pd.read_csv("02.Search1.csv")
search2 = pd.read_csv("03.Search2.csv")
session = pd.read_csv("05.Session.csv")

In [5]:
custom.head().to_csv("custom_head.csv", index=False)

In [7]:
master.head().to_csv("master_head.csv", index=False)

In [9]:
product.head().to_csv("product_head.csv", index=False)

In [11]:
session.head().to_csv("session.csv", index=False)

In [3]:
# 인테리어 브랜드들 제거 -> 벽지는 타일 형식으로 판매 수량이 기록되어, 판매 수량이 과도하게 많습니다.
print("인테리어 브랜드들 제거 전 product 개수  : {}".format(len(product)))
product  = product[product['PD_BRA_NM'] != '리드센스']
product  = product[product['PD_BRA_NM'] != '[리드센스]']
product  = product[product['PD_BRA_NM'] != '창데코']
product  = product[product['PD_BRA_NM'] != '코지샵']
product  = product[product['PD_BRA_NM'] != '[코지샵]']
product  = product[product['PD_BRA_NM'] != '바자르']
product  = product[product['PD_BRA_NM'] != '[바자르]']
product  = product[product['PD_BRA_NM'] != '한샘']
product  = product[product['PD_BRA_NM'] != '[한샘]']
product  = product[product['PD_BRA_NM'] != '24+']
product  = product[product['PD_BRA_NM'] != '[24+]']
product  = product[product['PD_BRA_NM'] != '창문그리기']
print("인테리어 브랜드들 제거 후 product 개수  : {}".format(len(product)))

인테리어 브랜드들 제거 전 product 개수  : 5024906
인테리어 브랜드들 제거 후 product 개수  : 5009812


In [9]:
# Str 형태로 되어 있는 PD_BUY_CT(수량), PD_BUY_AM(가격)은 
# int 형태의 데이터로 조정 ex) ‘1,440’ -> 1440

product['PD_BUY_CT'] =  product['PD_BUY_CT'].astype(str)
product['PD_BUY_CT'] = product['PD_BUY_CT'].replace(r',','',regex=True)
product['PD_BUY_CT'] =  product['PD_BUY_CT'].astype(int)
product['PD_BUY_AM'] =  product['PD_BUY_AM'].astype(str)
product['PD_BUY_AM'] = product['PD_BUY_AM'].replace(r',','',regex=True)
product['PD_BUY_AM'] =  product['PD_BUY_AM'].astype(int)
session['TOT_SESS_HR_V'] =  session['TOT_SESS_HR_V'].astype(str)
session['TOT_SESS_HR_V'] = session['TOT_SESS_HR_V'].replace(r',','',regex=True)
session['TOT_SESS_HR_V'] = session['TOT_SESS_HR_V'].astype(float)

In [5]:
#TOT_PAG_VIEW_CT, TOT_SESS_HR_V에서의 nan 값 제거
print("session 전처리 전 개수 : {}".format(len(session)))
session = session.dropna(axis=0).copy()
session = session[session['TOT_SESS_HR_V'].isnull() == False]
session[session['TOT_SESS_HR_V'].isnull() == True].count()

#TOT_PAG_VIEW_CT가 3회 미만이면 제거
session = session[session['TOT_PAG_VIEW_CT'] > 3]
print("session 전처리 후 개수 : {}".format(len(session)))

#datetime으로 활용하기 위해 20180401 -> 2018-04-01 형태로 변경
session["SESS_DT"] = session["SESS_DT"].astype(str)
session["SESS_DT"] = session["SESS_DT"].apply(lambda x : x[:4] + "-" + x[4:6] + "-" + x[-2:])

session 전처리 전 개수 : 2712907
session 전처리 후 개수 : 2688092


In [6]:
#dataframe들 merge 하는 과정입니다.

total = pd.merge(product, custom)

total = total.dropna(axis=0).copy()

total = pd.merge(total, session, on = ['CLNT_ID','SESS_ID'], how = 'left')

total_0 = total.copy()

total_0 = total.drop_duplicates('SESS_ID',keep='first')

total_0 = total_0.loc[:,['SESS_ID','HITS_SEQ']]

total_b = pd.merge(total, total_0, on=['SESS_ID','HITS_SEQ'])

total_c = total_b.dropna(axis=0).copy()

total_d = total_c.drop(['CLNT_ID','PD_ADD_NM','PD_BRA_NM','DVC_CTG_NM',
                        'ZON_NM','CITY_NM', "PD_BUY_AM"],axis=1)

total_e = pd.merge(total_d, master, on = "PD_C")

total_e.drop(columns = ["PD_NM", "CLAC1_NM", "CLAC2_NM"], inplace = True)

total_e.drop(columns = ["SESS_ID", "PD_C"], inplace = True)

#전처리 및 데이터 통합 완료
total_e.to_csv("./AfterPreprocessing.csv", index = False)

In [7]:
total_e.head()

,HITS_SEQ,PD_BUY_CT,CLNT_GENDER,CLNT_AGE,SESS_SEQ,SESS_DT,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,CLAC3_NM
0,13,2,F,40,172.0,2018-04-13,18.0,113.0,남성스포츠티셔츠
1,21,1,F,30,58.0,2018-06-13,20.0,144.0,남성스포츠티셔츠
2,94,1,F,30,55.0,2018-06-11,93.0,1371.0,남성스포츠티셔츠
3,129,1,F,30,710.0,2018-07-17,179.0,1199.0,남성스포츠티셔츠
4,55,1,F,30,772.0,2018-06-14,123.0,2804.0,남성스포츠티셔츠
